In [1]:
import numpy as np
import time
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import gzip

df = pd.read_table("SNV/MANIFEST.txt")
filelist = []
header = []
for file_name in df[0:33]["filename"]:
    filelist.append(file_name)

#for m in filelist:
 #   print(m)
with gzip.open("SNV/"+filelist[0],"rb") as f:
    print(f)
    i = 0
    while i <=4:
        header.append(f.readline())
        i += 1
    mf = pd.read_csv(f, delimiter = "\t")
    table = mf

<gzip _io.BufferedReader name='SNV/02747363-f04a-4ba6-a079-fe4f87853788/TCGA.UCS.mutect.02747363-f04a-4ba6-a079-fe4f87853788.DR-10.0.somatic.maf.gz' 0x1fd8d78b4f0>


In [2]:
#header

In [3]:
table.isna().sum().sum()

512307

In [4]:
table.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10449 entries, 0 to 10448
Data columns (total 120 columns):
 #    Column                         Non-Null Count  Dtype  
---   ------                         --------------  -----  
 0    Hugo_Symbol                    10449 non-null  object 
 1    Entrez_Gene_Id                 10449 non-null  int64  
 2    Center                         10449 non-null  object 
 3    NCBI_Build                     10449 non-null  object 
 4    Chromosome                     10449 non-null  object 
 5    Start_Position                 10449 non-null  int64  
 6    End_Position                   10449 non-null  int64  
 7    Strand                         10449 non-null  object 
 8    Variant_Classification         10449 non-null  object 
 9    Variant_Type                   10449 non-null  object 
 10   Reference_Allele               10449 non-null  object 
 11   Tumor_Seq_Allele1              10449 non-null  object 
 12   Tumor_Seq_Allele2             

In [5]:
m = table.isna().sum().tolist()
m.sort(reverse =True)
print(m)

[10449, 10449, 10449, 10449, 10449, 10449, 10449, 10449, 10449, 10449, 10449, 10449, 10449, 10449, 10449, 10449, 10449, 10449, 10439, 10416, 10364, 10349, 10347, 10224, 10007, 9970, 9970, 9970, 9970, 9970, 9908, 9848, 9848, 9583, 8701, 8672, 8587, 8312, 8312, 8312, 8312, 8312, 8312, 8312, 8312, 8312, 8040, 7821, 7821, 7821, 4855, 4461, 3906, 2533, 2186, 1447, 1446, 1446, 1315, 1314, 1077, 942, 689, 626, 621, 621, 515, 202, 202, 202, 85, 80, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [6]:
Hugo_Symbol = table.loc[:,["Hugo_Symbol"]]
Y = table.loc[:,["Entrez_Gene_Id"]].values #independent variables
X = table.drop(["Hugo_Symbol","Entrez_Gene_Id"], axis = 1) #dependent variables

In [7]:
Y.tolist()
Y

array([[  6536],
       [  1263],
       [199920],
       ...,
       [441531],
       [  9737],
       [114824]], dtype=int64)

In [8]:
X.values

array([['BI', 'GRCh38', 'chr1', ..., 'COSM6063461;COSM6063462', True,
        'Valid'],
       ['BI', 'GRCh38', 'chr1', ..., nan, True, 'Valid'],
       ['BI', 'GRCh38', 'chr1', ..., nan, True, 'Unknown'],
       ...,
       ['BI', 'GRCh38', 'chrX', ..., nan, True, 'Unknown'],
       ['BI', 'GRCh38', 'chrX', ..., nan, True, 'Unknown'],
       ['BI', 'GRCh38', 'chrX', ..., nan, True, 'Unknown']], dtype=object)

In [9]:
#Remove columns with all row values NaN
X.dropna(axis=1,how="all", inplace = True)
m = X.isna().sum().tolist()
m.sort(reverse =True)
print(m)

[10439, 10416, 10364, 10349, 10347, 10224, 10007, 9970, 9970, 9970, 9970, 9970, 9908, 9848, 9848, 9583, 8701, 8672, 8587, 8312, 8312, 8312, 8312, 8312, 8312, 8312, 8312, 8312, 8040, 7821, 7821, 7821, 4855, 4461, 3906, 2533, 2186, 1447, 1446, 1446, 1315, 1314, 1077, 942, 689, 626, 621, 621, 515, 202, 202, 202, 85, 80, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [10]:
#missing_ratio = []
#for i in m:
#    value = (i/10449)*100
#    missing_ratio.append(value)
#missing_ratio

In [12]:
#Handling NaN in float type cloumns that are below treshold 0.70(70%)#
def delete_nan_values(X):
    length = len(Y)
    thresh = length*0.70
    m = X[X.columns[X.dtypes == float]]
    m = m.dropna(axis=1,thresh=int(thresh)).fillna(m.mean())
    X = X.drop(X.columns[X.dtypes == float], axis=1)
    X = pd.concat([X,m], axis=1)
    return X

In [13]:
#Handling categorical values
def handle_categ_values(X):
    labelencoder_x = LabelEncoder()
    m = X.columns[X.dtypes == object]
    for n in m:
        X[n] = labelencoder_x.fit_transform(X[n])
    m = X.columns[X.dtypes == bool]
    for n in m:
        X[n] = labelencoder_x.fit_transform(X[n])    
    return X

In [14]:
X=delete_nan_values(X)
X=handle_categ_values(X)
X

,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,...,src_vcf_id,tumor_bam_uuid,normal_bam_uuid,case_id,GDC_FILTER,COSMIC,MC3_Overlap,GDC_Validation_Status,PICK,TSL
0,0,0,0,44008472,44008472,0,13,2,64,64,...,31,7,20,1,8,1620,1,1,1.0,1.000000
1,0,0,0,44803280,44803280,0,9,2,64,64,...,31,7,20,1,8,1825,1,1,1.0,1.000000
2,0,0,0,56740711,56740711,0,13,2,31,31,...,31,7,20,1,5,1825,1,0,1.0,1.000000
3,0,0,0,155233116,155233116,0,0,2,31,31,...,31,7,20,1,8,1825,0,0,1.0,1.000000
4,0,0,0,157581530,157581530,0,15,2,31,31,...,31,7,20,1,0,1825,1,0,1.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10444,0,0,22,18642139,18642139,0,13,2,31,31,...,5,26,45,5,8,1825,1,0,1.0,1.000000
10445,0,0,22,47201524,47201524,0,9,2,83,83,...,5,26,45,5,8,1825,1,0,1.0,1.000000
10446,0,0,22,77969434,77969434,0,9,2,83,83,...,5,26,45,5,8,1825,1,0,1.0,1.543215
10447,0,0,22,102657224,102657224,0,9,2,31,31,...,5,26,45,5,8,1825,1,0,1.0,2.000000


In [15]:
X.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10449 entries, 0 to 10448
Data columns (total 81 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Center                       10449 non-null  int32  
 1   NCBI_Build                   10449 non-null  int32  
 2   Chromosome                   10449 non-null  int32  
 3   Start_Position               10449 non-null  int64  
 4   End_Position                 10449 non-null  int64  
 5   Strand                       10449 non-null  int32  
 6   Variant_Classification       10449 non-null  int32  
 7   Variant_Type                 10449 non-null  int32  
 8   Reference_Allele             10449 non-null  int32  
 9   Tumor_Seq_Allele1            10449 non-null  int32  
 10  Tumor_Seq_Allele2            10449 non-null  int32  
 11  dbSNP_RS                     10449 non-null  int32  
 12  dbSNP_Val_Status             10449 non-null  int32  
 13  Tumor_Sample_Bar

In [16]:
m = X.isna().sum().tolist()
m.sort(reverse =True)
print(m)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [17]:
labelencoder_y= LabelEncoder()  
Y = labelencoder_y.fit_transform(Y)
Y

C:\Users\anjel\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


array([1695,  344, 6058, ..., 6697, 2491, 5458], dtype=int64)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X.values, Y, test_size = 0.30,random_state=42)

In [20]:
X_train

array([[ 0.,  0.,  7., ...,  0.,  1.,  1.],
       [ 0.,  0.,  7., ...,  0.,  1.,  5.],
       [ 0.,  0., 19., ...,  1.,  1.,  1.],
       ...,
       [ 0.,  0., 15., ...,  0.,  1.,  1.],
       [ 0.,  0., 11., ...,  0.,  1.,  1.],
       [ 0.,  0.,  2., ...,  0.,  1.,  1.]])

In [25]:
st_x= StandardScaler()
X_train= st_x.fit_transform(X_train)

In [26]:
X_train

array([[ 0.        ,  0.        , -0.51498828, ..., -0.58229712,
         0.        , -0.43041155],
       [ 0.        ,  0.        , -0.51498828, ..., -0.58229712,
         0.        ,  2.80988519],
       [ 0.        ,  0.        ,  1.15375171, ...,  1.71733633,
         0.        , -0.43041155],
       ...,
       [ 0.        ,  0.        ,  0.59750504, ..., -0.58229712,
         0.        , -0.43041155],
       [ 0.        ,  0.        ,  0.04125838, ..., -0.58229712,
         0.        , -0.43041155],
       [ 0.        ,  0.        , -1.21029661, ..., -0.58229712,
         0.        , -0.43041155]])